<a href="https://colab.research.google.com/github/SALMA55ASHRAF/AI_instant_tasks/blob/master/Number_of_Orders_Prediction_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("https://raw.githubusercontent.com/amankharwal/Website-data/master/supplement.csv")
df.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188340 entries, 0 to 188339
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ID             188340 non-null  object 
 1   Store_id       188340 non-null  int64  
 2   Store_Type     188340 non-null  object 
 3   Location_Type  188340 non-null  object 
 4   Region_Code    188340 non-null  object 
 5   Date           188340 non-null  object 
 6   Holiday        188340 non-null  int64  
 7   Discount       188340 non-null  object 
 8   #Order         188340 non-null  int64  
 9   Sales          188340 non-null  float64
dtypes: float64(1), int64(3), object(6)
memory usage: 14.4+ MB


In [3]:
df.isnull().sum()

ID               0
Store_id         0
Store_Type       0
Location_Type    0
Region_Code      0
Date             0
Holiday          0
Discount         0
#Order           0
Sales            0
dtype: int64

In [4]:
df.describe()

,Store_id,Holiday,#Order,Sales
count,188340.000000,188340.000000,188340.000000,188340.000000
mean,183.000000,0.131783,68.205692,42784.327982
std,105.366308,0.338256,30.467415,18456.708302
min,1.000000,0.000000,0.000000,0.000000
25%,92.000000,0.000000,48.000000,30426.000000
50%,183.000000,0.000000,63.000000,39678.000000
75%,274.000000,0.000000,82.000000,51909.000000
max,365.000000,1.000000,371.000000,247215.000000


#### Now let’s explore some of the important features from this dataset to know about the factors affecting the number of orders for supplements

In [5]:
import plotly.express as px
pie = df["Store_Type"].value_counts()
store = pie.index
orders = pie.values

fig = px.pie(df, values=orders, names=store)
fig.show()

### s1 store has the most orders

### The above figure shows the distribution of the number of orders received according to the store type. Now let’s have a look at the distribution of the number of orders, according to the location

In [6]:
pie2 = df["Location_Type"].value_counts()
location = pie2.index
orders = pie2.values

fig = px.pie(df, values=orders, names=location)
fig.show()

### The above figure shows the distribution of the number of orders received according to the location. Now let’s have a look at the distribution of the number of orders, according to the discount

In [7]:
pie3 = df["Discount"].value_counts()
discount = pie3.index
orders = pie3.values
fig = px.pie(df, values=orders, names=discount)
fig.show()

### According to the above figure, most people still buy supplements if there is no discount on them. Now let’s have a look at how holidays affect the number of orders

In [8]:
pie4 = df["Holiday"].value_counts()
holiday = pie4.index
orders = pie4.values

fig = px.pie(df, values=orders, names=holiday)
fig.show()

### According to the above figure, most of the people buy supplements in working days.

## now lets prepare our data for model

In [9]:
df["Discount"] = df["Discount"].map({"No": 0, "Yes": 1})
df["Store_Type"] = df["Store_Type"].map({"S1": 1, "S2": 2, "S3": 3, "S4": 4})
df["Location_Type"] = df["Location_Type"].map({"L1": 1, "L2": 2, "L3": 3, "L4": 4, "L5": 5})
df.dropna(inplace=True)

x = np.array(df[["Store_Type", "Location_Type", "Holiday", "Discount"]])
y = np.array(df["#Order"])

In [10]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x,
                                                y, test_size=0.2,
                                                random_state=42)

### Now I will be using the light gradient boosting regression algorithm to train the model
### why:

1.   List item
2.   List item



In [11]:
! pip install lightgbm
import lightgbm as ltb
model = ltb.LGBMRegressor()
model.fit(xtrain, ytrain)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013218 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15
[LightGBM] [Info] Number of data points in the train set: 150672, number of used features: 4
[LightGBM] [Info] Start training from score 68.163401


LGBMRegressor()

In [26]:
ypred = model.predict(xtest)
data = pd.DataFrame(data={"Predicted Orders": ypred.flatten()})
data["Actual Orders"] = ytest
data['diff']=data['Actual Orders']-data['Predicted Orders']
data.head()


99.98892639006205